In [1]:
!git clone https://github.com/MaciejZamorski/3d-AAE.git

Cloning into '3d-AAE'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 48 (delta 6), reused 5 (delta 5), pack-reused 34
Receiving objects: 100% (48/48), 556.72 KiB | 16.37 MiB/s, done.
Resolving deltas: 100% (13/13), done.


In [2]:
%cd 3d-AAE

/content/3d-AAE


In [3]:
!pip install -r requirements.txt

  Cloning https://github.com/szagoruyko/pyinn.git (to revision master) to /tmp/pip-req-build-bfftkgif
  Running command git clone --filter=blob:none --quiet https://github.com/szagoruyko/pyinn.git /tmp/pip-req-build-bfftkgif
  Resolved https://github.com/szagoruyko/pyinn.git to commit 948388e4ee585b23ed41d352fc8863ea868874ad
  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement torch==0.4.1 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1)
ERROR: No matching distribution found for torch==0.4.1


#add to train


```
import sys
sys.path.append('/content/3d-AAE/')
```



#change hyperparams arch to aae

```
"gpu": 0,
"arch": "aae",
```




#add this to train line 233
```
X_cpu = X[k].cpu()  # Move the tensor to the CPU
x = X_cpu[0].numpy()
y = X_cpu[1].numpy()
z = X_cpu[2].numpy()

# Use x, y, and z in the plot_3d_point_cloud function
fig = plot_3d_point_cloud(x, y, z, in_u_sphere=True, show=False, title=str(epoch))
```



If you have saved weights load them to colab

In [ ]:
!python experiments/train_aae.py --config settings/hyperparams.json

2023-11-16 07:59:40,325: DEBUG    Device variable: cuda
2023-11-16 07:59:40,325: DEBUG    Current CUDA device: 0
2023-11-16 07:59:40,431: INFO     NumExpr defaulting to 2 threads.
ShapeNet doesn't exist in root directory /Users/maciek/shapenet. Downloading...
Extracting...
2023-11-16 08:00:52,622: DEBUG    Selected all classes. Loaded 48809 samples.
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of

#change eval find_best

```
import sys
sys.path.append('/content/3d-AAE/')

#line 92 change
arch = import_module(f"models.{train_config['arch']}")
```

In [ ]:
!python evaluation/find_best_epoch_on_validation.py --config settings/hyperparams.json

2023-10-19 08:42:12,866: DEBUG    Evaluating JensenShannon divergences on validation set on all saved epochs.
2023-10-19 08:42:12,866: DEBUG    Testing epochs: [5, 10, 15, 20, 25, 30, 35]
2023-10-19 08:42:12,901: DEBUG    Device variable: cuda
2023-10-19 08:42:12,901: DEBUG    Current CUDA device: 0
2023-10-19 08:42:14,442: DEBUG    Selected all classes. Loaded 2870 samples.
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
2023-10-19 08:45:32,423: DEBUG    Epoch: 35 JSD:  0.647470 Time: 0:03:15.091105
2023-10-19 08:48:48,753: DEBUG    Epoch: 30 JSD:  0.647212 Time: 0:

In [ ]:
import zipfile
import os
folder_path = "/Users/maciek/test-results"
output_filename = "weights.zip"

with zipfile.ZipFile(output_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), folder_path))

In [ ]:
from google.colab import files

files.download(output_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Steps to improve

Adding he initialization


```
 def initialize_weights(self):
        for layer in self.model:
            if isinstance(layer, nn.Linear):
                # Apply He initialization to linear layers
                nn.init.kaiming_normal_(layer.weight, mode='fan_in', nonlinearity='relu')
                if layer.bias is not None:
                    nn.init.constant_(layer.bias, 0)
```




Discriminator update
```
class Discriminator(nn.Module):
    def __init__(self, config):
        super().__init__()

        self.z_size = config['z_size']
        self.use_bias = config['model']['D']['use_bias']
        self.relu_slope = config['model']['D']['relu_slope']
        self.dropout = config['model']['D']['dropout']

        self.model = nn.Sequential(
            nn.Linear(self.z_size, 512, bias=self.use_bias),
            nn.ReLU(inplace=True),
            nn.Dropout(p=self.dropout),

            nn.Linear(512, 512, bias=self.use_bias),
            nn.ReLU(inplace=True),
            nn.Dropout(p=self.dropout),

            nn.Linear(512, 128, bias=self.use_bias),
            nn.ReLU(inplace=True),
            nn.Dropout(p=self.dropout),

            nn.Linear(128, 64, bias=self.use_bias),
            nn.ReLU(inplace=True),
            nn.Dropout(p=self.dropout),

            nn.Linear(64, 1, bias=self.use_bias),
            nn.Sigmoid()  # Add sigmoid activation for binary classification
        )

        # Apply He initialization to linear layers
        self.initialize_weights()

    def initialize_weights(self):
        for layer in self.model:
            if isinstance(layer, nn.Linear):
                # Apply He initialization to linear layers
                nn.init.kaiming_normal_(layer.weight, mode='fan_in', nonlinearity='relu')
                if layer.bias is not None:
                    nn.init.constant_(layer.bias, 0)

    def forward(self, x):
        logit = self.model(x)
        return logit
```




Encoder update
```
class Encoder(nn.Module):
    def __init__(self, config):
        super().__init__()

        self.z_size = config['z_size']
        self.use_bias = config['model']['E']['use_bias']
        self.relu_slope = config['model']['E']['relu_slope']

        self.conv = nn.Sequential(
            nn.Conv1d(in_channels=3, out_channels=64, kernel_size=1, bias=self.use_bias),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(64),

            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=1, bias=self.use_bias),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(128),

            nn.Conv1d(in_channels=128, out_channels=256, kernel_size=1, bias=self.use_bias),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(256),

            nn.Conv1d(in_channels=256, out_channels=256, kernel_size=1, bias=self.use_bias),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(256),

            nn.Conv1d(in_channels=256, out_channels=512, kernel_size=1, bias=self.use_bias),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(512),
        )

        self.fc = nn.Sequential(
            nn.Linear(512, 256, bias=True),
            nn.ReLU(inplace=True),
        )

        self.mu_layer = nn.Linear(256, self.z_size, bias=True)
        self.std_layer = nn.Linear(256, self.z_size, bias=True)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu)

    def forward(self, x):
        output = self.conv(x)
        # Apply global average pooling
        output = output.mean(dim=2)
        logit = self.fc(output)
        mu = self.mu_layer(logit)
        logvar = self.std_layer(logit)
        z = self.reparameterize(mu, logvar)
        return z, mu, logvar
```



pyinn for earth mover distance

In [ ]:
!python experiments/train_aae.py --config settings/hyperparams.json

2023-11-16 09:49:25,362: DEBUG    Device variable: cuda
2023-11-16 09:49:25,362: DEBUG    Current CUDA device: 0
2023-11-16 09:49:25,432: INFO     NumExpr defaulting to 2 threads.
2023-11-16 09:49:27,193: DEBUG    Selected all classes. Loaded 48809 samples.
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create.

In [ ]:
!python evaluation/find_best_epoch_on_validation.py --config settings/hyperparams.json

2023-11-16 09:21:16,155: DEBUG    Evaluating JensenShannon divergences on validation set on all saved epochs.
2023-11-16 09:21:16,156: DEBUG    Testing epochs: [5, 10, 15]
2023-11-16 09:21:16,192: DEBUG    Device variable: cuda
2023-11-16 09:21:16,192: DEBUG    Current CUDA device: 0
2023-11-16 09:21:18,006: DEBUG    Selected all classes. Loaded 2870 samples.
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
2023-11-16 09:24:58,054: DEBUG    Epoch: 15 JSD:  0.661997 Time: 0:03:35.879738
2023-11-16 09:28:37,490: DEBUG    Epoch: 10 JSD:  0.678902 Time: 0:03:39.404301
202

"experiment_name": "experiment2",
"relu_slope": 0.4
updated arch
amsgrad True


In [ ]:
!python experiments/train_aae.py --config settings/hyperparams.json

Streaming output truncated to the last 5000 lines.
2023-11-16 10:37:02,986: DEBUG    [13: (468)] Loss_D: -0.2091 (GP:  0.5673) Loss_EG: 8.2166 (REC:  6.6542) Time: 0:01:47.272996
2023-11-16 10:37:02,986: DEBUG    --------------------
2023-11-16 10:37:03,210: DEBUG    [13: (469)] Loss_D: -0.3027 (GP:  0.5366) Loss_EG: 9.0040 (REC:  7.2692) Time: 0:01:47.496652
2023-11-16 10:37:03,210: DEBUG    --------------------
2023-11-16 10:37:03,435: DEBUG    [13: (470)] Loss_D: -0.2303 (GP:  0.5993) Loss_EG: 8.7482 (REC:  7.0997) Time: 0:01:47.722382
2023-11-16 10:37:03,436: DEBUG    --------------------
2023-11-16 10:37:03,662: DEBUG    [13: (471)] Loss_D: -0.1417 (GP:  0.5390) Loss_EG: 9.2036 (REC:  7.6417) Time: 0:01:47.948972
2023-11-16 10:37:03,662: DEBUG    --------------------
2023-11-16 10:37:03,888: DEBUG    [13: (472)] Loss_D: 0.1774 (GP:  0.7954) Loss_EG: 9.0039 (REC:  7.5211) Time: 0:01:48.175419
2023-11-16 10:37:03,889: DEBUG    --------------------
2023-11-16 10:37:04,113: DEBUG    [

In [ ]:
!python evaluation/find_best_epoch_on_validation.py --config settings/hyperparams.json

2023-11-16 10:46:55,622: DEBUG    Evaluating JensenShannon divergences on validation set on all saved epochs.
2023-11-16 10:46:55,622: DEBUG    Testing epochs: [5, 10, 15]
2023-11-16 10:46:55,660: DEBUG    Device variable: cuda
2023-11-16 10:46:55,660: DEBUG    Current CUDA device: 0
2023-11-16 10:46:57,109: DEBUG    Selected all classes. Loaded 2870 samples.
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
2023-11-16 10:50:41,099: DEBUG    Epoch: 15 JSD:  0.707177 Time: 0:03:38.463090
2023-11-16 10:54:22,717: DEBUG    Epoch: 10 JSD:  0.720302 Time: 0:03:41.575853
202

In [ ]:
!python experiments/train_aae.py --config settings/hyperparams.json

Streaming output truncated to the last 5000 lines.
2023-11-16 13:40:18,018: DEBUG    [19: (933)] Loss_D: 0.2199 (GP:  0.3497) Loss_EG: 0.2366 (REC:  0.0007) Time: 0:02:42.100565
2023-11-16 13:40:18,018: DEBUG    --------------------
2023-11-16 13:40:18,189: DEBUG    [19: (934)] Loss_D: 0.4701 (GP:  0.4349) Loss_EG: 0.3373 (REC:  0.0006) Time: 0:02:42.271928
2023-11-16 13:40:18,189: DEBUG    --------------------
2023-11-16 13:40:18,360: DEBUG    [19: (935)] Loss_D: 0.3597 (GP:  0.3176) Loss_EG: 0.2470 (REC:  0.0007) Time: 0:02:42.443117
2023-11-16 13:40:18,360: DEBUG    --------------------
2023-11-16 13:40:18,532: DEBUG    [19: (936)] Loss_D: 0.2656 (GP:  0.2637) Loss_EG: 0.1820 (REC:  0.0007) Time: 0:02:42.614970
2023-11-16 13:40:18,532: DEBUG    --------------------
2023-11-16 13:40:18,703: DEBUG    [19: (937)] Loss_D: 0.3368 (GP:  0.2465) Loss_EG: 0.2096 (REC:  0.0007) Time: 0:02:42.786466
2023-11-16 13:40:18,704: DEBUG    --------------------
2023-11-16 13:40:18,872: DEBUG    [19: 

In [ ]:
!python evaluation/find_best_epoch_on_validation.py --config settings/hyperparams.json

python3: can't open file '/content/evaluation/find_best_epoch_on_validation.py': [Errno 2] No such file or directory


trying new loss:
KLD loss


```
import torch
import torch.nn as nn

class KLDLoss(nn.Module):
    def __init__(self):
        super(KLDLoss, self).__init__()

    def forward(self, mu, logvar):
        # Compute KLD loss for a VAE
        kld_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        return kld_loss
```




Contrastive Divergence Loss
```
import torch
import torch.nn as nn

class ContrastiveDivergenceLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(ContrastiveDivergenceLoss, self).__init__()
        self.margin = margin

    def forward(self, x1, x2, y):
        # Calculate Euclidean distance between x1 and x2
        euclidean_distance = torch.sqrt(torch.sum((x1 - x2) ** 2, dim=-1))

        # Contrastive Divergence Loss
        loss = y * euclidean_distance + (1 - y) * torch.clamp(self.margin - euclidean_distance, min=0)

        # Average over the batch
        loss = torch.mean(loss)

        return loss
```



In [57]:
!python experiments/train_aae.py --config settings/hyperparams.json

2023-11-17 19:26:56,419: DEBUG    Device variable: cuda
2023-11-17 19:26:56,419: DEBUG    Current CUDA device: 0
2023-11-17 19:26:56,489: INFO     NumExpr defaulting to 2 threads.
2023-11-17 19:26:56,798: DEBUG    Selected car classes. Loaded 6372 samples.
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. 

In [58]:
!python evaluation/find_best_epoch_on_validation.py --config settings/hyperparams.json

2023-11-17 19:29:30,091: DEBUG    Evaluating JensenShannon divergences on validation set on all saved epochs.
2023-11-17 19:29:30,092: DEBUG    Testing epochs: [5, 10, 15, 20, 25, 30, 35, 40]
2023-11-17 19:29:30,128: DEBUG    Device variable: cuda
2023-11-17 19:29:30,129: DEBUG    Current CUDA device: 0
2023-11-17 19:29:30,185: DEBUG    Selected car classes. Loaded 375 samples.
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
2023-11-17 19:29:59,976: DEBUG    Epoch: 40 JSD:  0.377993 Time: 0:00:27.010999
2023-11-17 19:30:27,072: DEBUG    Epoch: 35 JSD:  0.385490 Time:

In [52]:
import zipfile
import os
folder_path = "/Users/maciek/test-results/aae/experiment_car2"
output_filename = "weights.zip"

with zipfile.ZipFile(output_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), folder_path))

In [ ]:
from google.colab import files

files.download(output_filename)